<a href="https://colab.research.google.com/github/LukeWeidenwalker/ai-notebooks/blob/master/fastai_mnist.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -Uqq fastbook
import fastbook
fastbook.setup_book()

     |████████████████████████████████| 727kB 14.9MB/s 
     |████████████████████████████████| 1.2MB 42.8MB/s 
     |████████████████████████████████| 51kB 8.3MB/s 
     |████████████████████████████████| 194kB 61.2MB/s 
     |████████████████████████████████| 61kB 9.9MB/s 
Mounted at /content/gdrive


In [2]:
from fastai.vision.all import *
from fastbook import *

In [3]:
path = untar_data(URLs.MNIST_SAMPLE)
Path.BASE_PATH = path

In [4]:
(path/"train").ls()
# what I need to do:
# I need to load in the dataset as tensors, and create a corresponding y tensor with the right labels

(#2) [Path('train/7'),Path('train/3')]

In [5]:
threes = torch.stack([tensor(Image.open(im)) for im in (path/"train"/'3').ls().sorted()])
sevens = torch.stack([tensor(Image.open(im)) for im in (path/"train"/'7').ls().sorted()])
threes.shape, sevens.shape

(torch.Size([6131, 28, 28]), torch.Size([6265, 28, 28]))

In [6]:
x_train = torch.cat((threes, sevens)).view(-1, 28*28) / 255.0; x_train.shape
x_train.shape, x_train.type()

(torch.Size([12396, 784]), 'torch.FloatTensor')

In [7]:
y_train = tensor([0]*len(threes) + [1]*len(sevens)).unsqueeze(1); y_train.shape, y_train.type()

(torch.Size([12396, 1]), 'torch.LongTensor')

In [8]:
train_dset = list(zip(x_train, y_train))

In [9]:
# Now do the same for the validation set
threes_valid = torch.stack([tensor(Image.open(im)) for im in (path/"valid"/'3').ls().sorted()])
sevens_valid = torch.stack([tensor(Image.open(im)) for im in (path/"valid"/'7').ls().sorted()])

x_valid = torch.cat((threes_valid, sevens_valid)).view(-1, 28*28) / 255.0; x_train.shape
y_valid = tensor([0]*len(threes_valid) + [1]*len(sevens_valid)).unsqueeze(1); y_valid.shape, y_valid.type()

valid_dset = list(zip(x_valid, y_valid))

In [10]:
train_dl = DataLoader(train_dset, shuffle=True, bs=256)
valid_dl = DataLoader(valid_dset, shuffle=False, bs=256)
dls = DataLoaders(train_dl, valid_dl)

In [11]:
dls.train.one_batch()[0].shape

torch.Size([256, 784])

# Model

In [59]:
def mnist_loss(preds, target):
  return torch.where(target==1, 1-preds, preds).mean()

preds = tensor([0.2, 0.6, 0.3])
target = tensor([0, 1, 1])
mnist_loss(preds, target)

tensor(0.4333)

In [38]:
def batch_accuracy(preds, yb):
  preds = preds.sigmoid()
  correct = (preds > 0.5) == yb
  return correct.float().mean()

xb = tensor([0.2, 0.2, 0.6])
yb = tensor([0, 1, 1])
batch_accuracy(xb, yb)

tensor(0.6667)

In [39]:
class BasicOptimiser:
  def __init__(self, params, lr):
    self.params,self.lr = list(params),lr

  def step(self):
    for p in self.params:
      p.data -= p.grad.data * self.lr

  def zero_grad(self):
    for p in self.params:
      p.grad = None

In [40]:
class SimpleNet:
  def __init__(self, input_size, output_size):
    self.w = self.init_parameters((input_size, output_size))
    self.b = self.init_parameters(1)

  def init_parameters(self, shape):
    return torch.rand(shape).requires_grad_()

  def __call__(self, xb):
    return xb@self.w + self.b

  def parameters(self):
    return self.w, self.b

In [41]:
class BasicSigmoid:
  def __call__(self, xb):
    return xb.sigmoid()

  def parameters(self):
    return []

In [42]:
class BasicSequential:
  def __init__(self, layers):
    self.layers = layers

  def __call__(self, xb):
    for layer in self.layers:
      xb = layer(xb)
    return xb

  def parameters(self):
    return [p for layer in self.layers for p in layer.parameters()]
    

In [60]:
class BasicLearner:
  def __init__(self, dls, model, opt_func, loss_function, batch_accuracy):
    self.dls = dls
    self.model = model
    self.opt_func = opt_func
    self.loss_function = loss_function
    self.batch_accuracy = batch_accuracy

  def validate_epoch(self):
    accs = [self.batch_accuracy(self.model(xb), yb) for xb, yb in self.dls.valid]
    return round(torch.stack(accs).mean().item(), 4)

  def fit(self, epochs):
    for epoch in range(epochs):
      accs = []
      for xb, yb in self.dls.train:
        preds = self.model(xb)
        loss = self.loss_function(preds, yb)
        loss.backward()
        self.opt_func.step()
        self.opt_func.zero_grad()
      # print(model.parameters()[0])
      # print(model.parameters()[0].grad)
      print(f"Epoch {epoch}, Accuracy: {self.validate_epoch()}")
      

In [61]:
model = SimpleNet(28*28, 1)

In [62]:
learner = BasicLearner(dls, model, BasicOptimiser(model.parameters(), 0.03), mnist_loss, batch_accuracy)
learner.fit(10)

Epoch 0, Accuracy: 0.5068
Epoch 1, Accuracy: 0.5068
Epoch 2, Accuracy: 0.5503
Epoch 3, Accuracy: 0.7471
Epoch 4, Accuracy: 0.8506
Epoch 5, Accuracy: 0.9072
Epoch 6, Accuracy: 0.9268
Epoch 7, Accuracy: 0.9414
Epoch 8, Accuracy: 0.9487
Epoch 9, Accuracy: 0.9541
